# Paintera

In this example, we will demonstrate how to view 3D data in the proof-reading and ground truth annotation tool [Paintera](http://github.com/saalfeldlab/paintera). 3D representations of label data are generated on the fly. This notebook depends on the highly experimental [paynter](https://github.com/saalfeldlab/payntera) Python package:
```
pip install git+https://github.com/saalfeldlab/payntera
```

First, create some arbitrary `numpy.ndarray` raw and label data.

In [1]:
import numpy as np
import scipy.ndimage

shape      = (80,80,50)
x, y, z    = np.indices(shape)
fx, fy, fz = 2 * np.pi / np.array(shape) * np.array([10, 1, 3])

raw        = (1+np.sin(x * fx)) * (1+np.sin(y * fy)) * (1+x*y/(shape[0]*shape[1]))**2 * (1+np.cos(z * fz)) * ((x+y+z)/np.sum(shape))
labels, nb = scipy.ndimage.label(raw > 0.5)

Then, convert data into ImgLib2 data structures and add to a Paintera instance

In [2]:
# set heap size to reasonable value
import jnius_config
jnius_config.add_options('-Xmx2g')

import os
os.environ['PAINTERA_VERSION'] = '0.7.0'

import payntera
import payntera.jfx

# imglyb and jnius must be imported after payntera is imported!
import imglyb
# jnius must be imported after imglyb is imported!
from jnius import autoclass, JavaException

payntera.jfx.init_platform()

In [3]:
raw_img    = imglyb.to_imglib(raw)
labels_img = imglyb.to_imglib(labels)

PainteraBaseView = autoclass('org.janelia.saalfeldlab.paintera.PainteraBaseView')
viewer           = PainteraBaseView.defaultView()
pbv              = viewer.baseView
scene, stage     = payntera.jfx.start_stage(viewer.paneWithStatus.getPane())


raw_state = pbv.addSingleScaleRawSource(raw_img, [1.0, 1.0, 1.0], [0.0, 0.0, 0.0], np.min(raw), 7, 'blub')
try:
    state     = pbv.addSingleScaleLabelSource(labels_img, [1.0, 1.0, 1.0], [0.0, 0.0, 0.0], nb+1, 'bla')
except JavaException as e:
    for st in e.stacktrace:
        print(st)
    raise e

viewer.keyTracker.installInto(scene)
scene.addEventFilter(autoclass('javafx.scene.input.MouseEvent').ANY, viewer.mouseTracker)